# Split data

In [88]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import collections

# Load the dataset
data_dir = 'data'
# data = pd.read_csv(f'{data_dir}/audio_features_no_duplicates_new_normalized.csv')
data = pd.read_csv(f'{data_dir}/audio_features_no_duplicates_new.csv')

In [89]:
# Exclude the first column (filename) and assume the last column is the label
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values

# Check the distribution of labels
label_distribution = collections.Counter(y)
print(f"Label distribution: {label_distribution}")

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

Label distribution: Counter({0: 1752, 1: 1678})


# Random Forrest Model

In [90]:
from sklearn.ensemble import RandomForestClassifier

# Train the classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)

In [91]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 83.89%
Precision: 0.84
Recall: 0.84
F1 Score: 0.84
Confusion Matrix:
[[583 114]
 [107 568]]
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       697
           1       0.83      0.84      0.84       675

    accuracy                           0.84      1372
   macro avg       0.84      0.84      0.84      1372
weighted avg       0.84      0.84      0.84      1372



In [77]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

df = pd.read_csv('data/audio_features_no_duplicates_new.csv')

# Select the feature columns (excluding 'filename' and 'label')
feature_columns = df.columns.difference(['filename', 'label'])

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
scaler.fit_transform(df[feature_columns])

array([[0.49188227, 0.5667704 , 0.48453651, ..., 0.22208931, 0.38692114,
        0.18086719],
       [0.5200094 , 0.20566048, 0.17318478, ..., 0.33800361, 0.41868654,
        0.3264133 ],
       [0.30905682, 0.6230946 , 0.58578646, ..., 0.29127289, 0.35138036,
        0.18888783],
       ...,
       [0.66727867, 0.58355937, 0.61037651, ..., 0.13027951, 0.32597473,
        0.33320646],
       [0.44947912, 0.44832641, 0.51268423, ..., 0.25652767, 0.52165061,
        0.29354363],
       [0.60728603, 0.52606   , 0.4116936 , ..., 0.16556209, 0.33786586,
        0.46637179]])

In [93]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import librosa
import numpy as np

def extract_features(file_path):
    
    # Load the audio file
    y, sr = librosa.load(file_path)

    # Extract features
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr).T, axis=0)
    zero_crossings = np.mean(librosa.feature.zero_crossing_rate(y))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    rms_energy = np.mean(librosa.feature.rms(y=y))

    # Combine all features into a single array
    features = np.hstack([mfccs, chroma, spectral_contrast, zero_crossings, spectral_bandwidth, rms_energy])

    return features

def predict_track(file_path, model, scaler):
    # Extract features from the audio file
    features = extract_features(file_path)
    
    # Convert features to a DataFrame with the same columns as used in training
    # feature_df = pd.DataFrame([features], columns=scaler.feature_names_in_)
    feature_df = pd.DataFrame([features])

    # Scale the features
    # features_scaled = scaler.transform(feature_df)
    
    # Make a prediction and get probabilities
    # probabilities = model.predict_proba(features_scaled)
    probabilities = model.predict_proba(feature_df)
    
    return probabilities

# Example usage
proper_track_directory = 'data/Mike Parker - Rainmaker.mp3'
# non_proper_track_directory = 'data/Mark Dekoda - Rave Harder Techno Bass.mp3'

proper_track_path = os.path.join(proper_track_directory)
# non_proper_track_path = os.path.join(non_proper_track_directory)

# Assuming clf is your trained RandomForestClassifier and scaler is your MinMaxScaler
proper_prediction = predict_track(proper_track_path, clf, scaler)
# non_proper_prediction = predict_track(non_proper_track_path, clf, scaler)

print(f"Prediction for {proper_track_directory}: {proper_prediction}")
# print(f"Prediction for {non_proper_track_directory}: {non_proper_prediction}")

Prediction for data/Mike Parker - Rainmaker.mp3: [[0.06 0.94]]


In [6]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred))
print("XGBoost Classification Report:\n", classification_report(y_test, y_pred))

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ['dlopen(/Users/zein/opt/miniconda3/envs/proper-classifier/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib, 6): no suitable image found.  Did find:\n\t/Users/zein/opt/miniconda3/envs/proper-classifier/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib: mach-o, but wrong architecture\n\t/Users/zein/opt/miniconda3/envs/proper-classifier/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib: mach-o, but wrong architecture']


# Spectrogram based CNN Model

## Training the CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes: proper and improper, so 2 output neurons
])

#Load images and prepare the training and testing data


# Preprocess your data
# Assuming X_train and X_test are your image data and y_train, y_test are your labels
# Ensure X_train and X_test are reshaped to (num_samples, 128, 128, 3) and normalized
X_train = X_train.reshape(-1, 128, 128, 3) / 255.0
X_test = X_test.reshape(-1, 128, 128, 3) / 255.0

# Convert labels to categorical if they are not already
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
